In [1]:
# Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, insert, select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, Numeric, Text, Float

## generate SQLite database and tables

In [2]:
# Define an engine to create sqlite database
engine = create_engine('sqlite:///hawaii.sqlite')

In [3]:
# Initialize MetaData: metadata
metadata = MetaData()

In [4]:
# Build a table: station
station = Table('station', metadata,
               Column('id', Integer(), primary_key=True),
               Column('station', String(255)),
               Column('name', String(255)),
               Column('latitude', Float()),
               Column('longitude', Float()),
               Column('elevation', Float()))

In [5]:
# Build a table: measurement
measurement = Table('measurement', metadata,
               Column('id', Integer(), primary_key=True),
               Column('station', String(255)),
               Column('date', String(30)),
               Column('prcp', Float()),
               Column('tobs', Integer()))

In [6]:
# Create the table in the database
metadata.create_all(engine)

In [7]:
print(engine.table_names())

['measurement', 'station']


## input information to station table from CSV file

In [8]:
station_df = pd.read_csv("hawaii_stations.csv")

In [9]:
# Create an empty list: values_list
values_list = []

# Iterate over the rows
for index, row in station_df.iterrows():
    # Create a dictionary with the values
    data = {'station': row[0], 'name': row[1], 'latitude':row[2],
            'longitude': row[3],'elevation': row[4]}
    # Append the dictionary to the values list
    values_list.append(data)

In [10]:
# Build insert statement: stmt
stmt = insert(station)

# Use values_list to insert data: results
connection = engine.connect()
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)

9


### test the table

In [11]:
# Reflect station table via engine:
station = Table('station', metadata, autoload=True, autoload_with=engine)

# Build select statement for table: stmt
stmt = select([station])

# Print the emitted statement to see the SQL emitted
print(stmt)
print('----------------')

# Execute the statement and print the results
results = connection.execute(stmt).fetchall()
print(results)

SELECT station.id, station.station, station.name, station.latitude, station.longitude, station.elevation 
FROM station
----------------
[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6), (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0), (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9), (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6), (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5), (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9), (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9), (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]


## input information to measurement table from CSV file

In [12]:
measurement_df = pd.read_csv("clean_hawaii_measurements.csv")

# Create an empty list: values_list
values_list = []

# Iterate over the rows
for index, row in measurement_df.iterrows():
    # Create a dictionary with the values
    data = {'station': row[0], 'date': row[1], 'prcp':row[2], 'tobs': row[3]}
    # Append the dictionary to the values list
    values_list.append(data)

# Build insert statement: stmt
stmt = insert(measurement)

# Use values_list to insert data: results
connection = engine.connect()
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)

19550


### test the table

In [14]:
# Reflect station table via engine:
station = Table('measurement', metadata, autoload=True, autoload_with=engine)

# Build select statement for table: stmt
stmt = select([measurement]).limit(10)

# Print the emitted statement to see the SQL emitted
print(stmt)
print('----------------')

# Execute the statement and print the results
results = connection.execute(stmt).fetchall()
print(results)

SELECT measurement.id, measurement.station, measurement.date, measurement.prcp, measurement.tobs 
FROM measurement
 LIMIT :param_1
----------------
[(1, 'USC00519397', '2010-01-01', 0.08, 65), (2, 'USC00519397', '2010-01-02', 0.0, 63), (3, 'USC00519397', '2010-01-03', 0.0, 74), (4, 'USC00519397', '2010-01-04', 0.0, 76), (5, 'USC00519397', '2010-01-06', 0.0, 73), (6, 'USC00519397', '2010-01-07', 0.06, 70), (7, 'USC00519397', '2010-01-08', 0.0, 64), (8, 'USC00519397', '2010-01-09', 0.0, 68), (9, 'USC00519397', '2010-01-10', 0.0, 73), (10, 'USC00519397', '2010-01-11', 0.01, 64)]
